# Image Preprocessing

In [1]:
pwd

'/home/wsuser/work'

In [2]:
!pip install keras
!pip install tensorflow

In [6]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='sfAD6_eISTteP-XkAHUXxnZ95lpjeR-C4U5lDsK6EGv8',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'humannailimageprocessing-donotdelete-pr-zjzfz9oh8y0f4d'
object_key = 'Dataset.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [7]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()), 'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [8]:
# Importing Libraries

from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

In [9]:
# Loading Model

vgg = VGG16(input_shape=[64,64] + [3], weights = 'imagenet', include_top = False)

58900480/58889256 [==============================] - 0s 0us/step


In [10]:
# don't train existing weights

for layer in vgg.layers:
    layer.trainable = False
    
# our layers
x = Flatten()(vgg.output)

In [11]:
# Adding Output layer

prediction = Dense(17, activation = 'softmax')(x) 

In [12]:
# creating a model object

model = Model(inputs = vgg.input, outputs = prediction)

In [13]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 16, 16, 128)       0     

In [14]:
# Compile the model

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'], run_eagerly = True)

In [15]:
# importing datagen library

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [16]:
train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [18]:
# Loading train and test set

X_train = train_datagen.flow_from_directory(r"/home/wsuser/work/Dataset/train", target_size = (64, 64), batch_size = 32, class_mode = 'categorical')
X_test = test_datagen.flow_from_directory(r"/home/wsuser/work/Dataset/test", target_size = (64, 64), batch_size = 32, class_mode = 'categorical')

Found 655 images belonging to 17 classes.
Found 183 images belonging to 17 classes.


In [19]:
# checking indices

X_train.class_indices

{'Darier_s disease': 0,
 'Muehrck-e_s lines': 1,
 'aloperia areata': 2,
 'beau_s lines': 3,
 'bluish nail': 4,
 'clubbing': 5,
 'eczema': 6,
 'half and half nailes (Lindsay_s nails)': 7,
 'koilonychia': 8,
 'leukonychia': 9,
 'onycholycis': 10,
 'pale nail': 11,
 'red lunula': 12,
 'splinter hemmorrage': 13,
 'terry_s nail': 14,
 'white nail': 15,
 'yellow nails': 16}

In [20]:
# Fiiting the model

model.fit_generator(X_train, steps_per_epoch = len(X_train)//3, epochs = 200, validation_data = X_test, validation_steps = len(X_test)//3)

/tmp/wsuser/ipykernel_164/1927681411.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(X_train, steps_per_epoch = len(X_train)//3, epochs = 200, validation_data = X_test, validation_steps = len(X_test)//3)


Epoch 1/200
7/7 [==============================] - 37s 5s/step - loss: 2.9322 - accuracy: 0.0670 - val_loss: 2.7968 - val_accuracy: 0.1250
Epoch 2/200
7/7 [==============================] - 35s 5s/step - loss: 2.7378 - accuracy: 0.1027 - val_loss: 2.7077 - val_accuracy: 0.1094
Epoch 3/200
7/7 [==============================] - 37s 5s/step - loss: 2.7036 - accuracy: 0.1205 - val_loss: 2.7035 - val_accuracy: 0.1250
Epoch 4/200
7/7 [==============================] - 36s 5s/step - loss: 2.5777 - accuracy: 0.1875 - val_loss: 2.6305 - val_accuracy: 0.2188
Epoch 5/200
7/7 [==============================] - 35s 5s/step - loss: 2.6187 - accuracy: 0.1741 - val_loss: 2.5921 - val_accuracy: 0.2344
Epoch 6/200
7/7 [==============================] - 32s 5s/step - loss: 2.4453 - accuracy: 0.2754 - val_loss: 2.5305 - val_accuracy: 0.2500
Epoch 7/200
7/7 [==============================] - 33s 5s/step - loss: 2.3903 - accuracy: 0.2995 - val_loss: 2.4943 - val_accuracy: 0.1562
Epoch 8/200
7/7 [==========

7/7 [==============================] - 38s 5s/step - loss: 1.0970 - accuracy: 0.7391 - val_loss: 1.0862 - val_accuracy: 0.6875
Epoch 118/200
7/7 [==============================] - 35s 5s/step - loss: 1.0942 - accuracy: 0.6786 - val_loss: 0.9471 - val_accuracy: 0.7812
Epoch 119/200
7/7 [==============================] - 32s 4s/step - loss: 1.0761 - accuracy: 0.7150 - val_loss: 0.8909 - val_accuracy: 0.8281
Epoch 120/200
7/7 [==============================] - 40s 6s/step - loss: 1.0599 - accuracy: 0.7277 - val_loss: 0.9214 - val_accuracy: 0.7812
Epoch 121/200
7/7 [==============================] - 35s 5s/step - loss: 1.0772 - accuracy: 0.7005 - val_loss: 0.9682 - val_accuracy: 0.7812
Epoch 122/200
7/7 [==============================] - 34s 5s/step - loss: 0.9652 - accuracy: 0.7812 - val_loss: 0.9803 - val_accuracy: 0.7812
Epoch 123/200
7/7 [==============================] - 37s 5s/step - loss: 1.0822 - accuracy: 0.7391 - val_loss: 1.0747 - val_accuracy: 0.7656
Epoch 124/200
7/7 [========

In [21]:
model.save("Vgg-16-nail-disease.h5")

In [22]:
!tar -zcvf human_nail_image_model.tgz Vgg-16-nail-disease.h5

Vgg-16-nail-disease.h5


In [23]:
!pip install watson-machine-learning-client

     |████████████████████████████████| 538 kB 17.8 MB/s eta 0:00:01


In [25]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
    "url" : "https://us-south.ml.cloud.ibm.com",
    "apikey" : "nE_NMPr3xyuxZ1YZ2vwq3yieoYE2LJe6RDMFXd8N6ZcC"
}
client = APIClient(wml_credentials)

In [26]:
def guid_space_name(client, human_nail_image_processing_space):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name'] == human_nail_image_processing_space)['metadata']['id'])

In [27]:
client.spaces.get_details()

{'resources': [{'entity': {'compute': [{'crn': 'crn:v1:bluemix:public:pm-20:us-south:a/e4de7d7e6954440d81939b4328ea39e5:d0d10823-52a3-4952-a6f5-cd557788caae::',
      'guid': 'd0d10823-52a3-4952-a6f5-cd557788caae',
      'name': 'Watson Machine Learning-ym',
      'type': 'machine_learning'}],
    'description': '',
    'name': 'human_nail_image_processing_space',
    'scope': {'bss_account_id': 'e4de7d7e6954440d81939b4328ea39e5'},
    'stage': {'production': False},
    'status': {'state': 'active'},
    'storage': {'properties': {'bucket_name': '89040bb3-5671-478e-8476-9ea73521698b',
      'bucket_region': 'us-south',
      'credentials': {'admin': {'access_key_id': '8384899c3afe4f048b4aed61441b884a',
        'api_key': 'bMUZGBujGhiMHdokq4o_H3aYEdKZjL8j-eQ55p_TRQnm',
        'secret_access_key': 'e7e908f9b305a2d0c71289b658517535ee680dde269bfc4d',
        'service_id': 'ServiceId-4aa338f3-1db6-46f6-b426-c593f1450dce'},
       'editor': {'access_key_id': '67e29c02155a4f5b840bb9ce890763

In [28]:
space_id = guid_space_name(client, 'human_nail_image_processing_space')
space_id

'37602227-6637-477d-94e0-cd544b902c4b'

In [29]:
client.set.default_space(space_id)

'SUCCESS'

In [30]:
client.software_specifications.list(100)

-------------------------------  ------------------------------------  ----
NAME                             ASSET_ID                              TYPE
default_py3.6                    0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12        020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt       069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6          09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12       09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9        0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6            0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                       0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod     1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6                10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl        111e41b3-de2d-5422-a4d6-bf776828c4b7  base
runtime-22.1

In [31]:
software_space_id = client.software_specifications.get_uid_by_name('tensorflow_rt22.1-py3.9')
software_space_id

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [32]:
model_details = client.repository.store_model(model = 'human_nail_image_model.tgz', meta_props = {
    client.repository.ModelMetaNames.NAME:"CNN Model Buiding",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_space_id
})

In [33]:
model_id = client.repository.get_model_id(model_details)
model_id

'0de26764-fdce-4316-bebe-1023de1567ca'

In [36]:
pwd


'/home/wsuser/work'